# Lab 1: Fast Agent - Construa um Agente SQL Rapidamente!

<img src="./assets/LC_L1_top.png" align="left" width="500">

## 🎯 O que você vai aprender neste Lab

Neste lab introdutório, você vai construir um **agente SQL completo em poucas linhas de código**. Este é o ponto de partida do curso LangChain Essentials.

### Conceitos-chave:
- **LangChain**: Biblioteca open-source para construir aplicações com LLMs (Large Language Models)
- **Agente ReAct**: Agente que opera em ciclo de **Reasoning** (raciocínio) e **Acting** (ação)
- **Tools (Ferramentas)**: Funções que o agente pode usar para interagir com o mundo real

### Por que usar LangChain?

1. **Independência de modelo**: Troque entre OpenAI, Anthropic, Databricks sem reescrever código
2. **Agentes poderosos**: Framework ReAct built-in com persistência e durabilidade
3. **Ecossistema rico**: Mais de 100 integrações com provedores de LLM
4. **Middleware customizável**: Adapte o agente às suas necessidades específicas

## 🔄 O Ciclo ReAct (Reasoning + Acting)

O agente ReAct opera em um loop contínuo:

```
┌─────────────────────────────────────────────────────┐
│                    CICLO ReAct                       │
│                                                      │
│   1. RECEBE PERGUNTA                                │
│          ↓                                          │
│   2. RACIOCINA (Reasoning)                          │
│          ↓                                          │
│   3. USA FERRAMENTA (Acting)                        │
│          ↓                                          │
│   4. OBSERVA RESULTADO                              │
│          ↓                                          │
│   5. Pode responder? ────→ SIM ──→ RESPOSTA FINAL  │
│          │                                          │
│          └── NÃO ──→ Volta ao passo 2              │
└─────────────────────────────────────────────────────┘
```

**Importante**: O agente pode errar e se autocorrigir! Quando uma query SQL falha, ele analisa o erro e tenta novamente com uma abordagem diferente.

## ⚙️ Setup - Configuração do Ambiente

Primeiro, instalamos as bibliotecas necessárias e configuramos o MLflow para rastreamento automático.

**Bibliotecas utilizadas:**
- `mlflow`: Rastreamento de experimentos e modelos
- `langchain`: Framework principal para LLMs
- `databricks-langchain`: Integração nativa com Databricks
- `tabulate`: Formatação de tabelas para saída

In [0]:
%pip install -U mlflow>=3 langchain>=1 langchain.community tabulate databricks-langchain --upgrade --force-reinstall typing_extensions
dbutils.library.restartPython()

In [0]:
import mlflow

In [0]:
mlflow.langchain.autolog()

### 📊 MLflow Autolog

O `mlflow.langchain.autolog()` registra automaticamente todas as chamadas do agente, incluindo:
- Inputs e outputs de cada invocação
- Latência de cada chamada
- Tokens utilizados
- Histórico de mensagens

Isso é essencial para **debugging**, **monitoramento** e **avaliação** de agentes em produção.

In [0]:
%sql
show catalogs

## 🔨 Definindo a Ferramenta (Tool)

Uma **tool** é uma função Python que o agente pode chamar para interagir com o mundo externo. O decorador `@tool` transforma a função em uma ferramenta que o LLM pode descobrir e usar.

### Anatomia de uma Tool:
1. **Nome**: Inferido do nome da função (`execute_sql`)
2. **Descrição**: Extraída da docstring - o LLM usa isso para decidir QUANDO usar a ferramenta
3. **Parâmetros**: Tipos inferidos da assinatura - o LLM usa isso para saber COMO chamar a ferramenta
4. **Retorno**: String que será enviada de volta ao LLM como "observação"

In [0]:
from langchain_core.tools import tool

@tool
def execute_sql(query: str) -> str:
    """Executa um comando SQL no Databricks e retorna os resultados."""
    try:
        result = spark.sql(query).limit(5).toPandas()
        return result.to_markdown()
    except Exception as e:
        return f"Error: {e}"

## 📝 System Prompt - Definindo o Comportamento do Agente

O **system prompt** é a mensagem mais importante do agente. Ele define:
- **Persona**: Quem o agente "é" (ex: analista SQL)
- **Regras**: O que pode e não pode fazer
- **Comportamento**: Como deve reagir a erros e situações específicas

### Dicas para System Prompts Eficazes:
1. Seja **claro e específico** sobre o papel do agente
2. Defina **limites** explícitos (somente SELECT, máximo de linhas)
3. Inclua instruções de **autocorreção** ("se erro, tente novamente")
4. Mantenha **completo mas conciso** - prompts muito longos podem confundir

In [0]:
SYSTEM_PROMPT = """Você é um analista cuidadoso de Databricks SQL.
Regras:
- Pense passo a passo.
- Quando precisar de dados, chame a ferramenta `execute_sql` com UMA query SELECT.
- Apenas leitura; sem INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limite a 5 linhas de saída, a menos que o usuário peça mais.
- Se a ferramenta retornar 'Error:', revise o SQL e tente novamente.
- Prefira listas explícitas de colunas; evite SELECT *.
"""

## 🤖 Criando o Agente

Agora juntamos todas as peças para criar o agente:

1. **Model (llm)**: O cérebro - `ChatDatabricks` conecta ao endpoint de serving
2. **Tools**: As mãos - lista de ferramentas disponíveis
3. **System Prompt**: A personalidade - define comportamento

### Parâmetros importantes:
- `temperature=0.1`: Baixa temperatura = respostas mais determinísticas e precisas (ideal para SQL)
- `endpoint`: Nome do modelo servido no Databricks Model Serving

In [0]:
from databricks_langchain import ChatDatabricks
from langchain.agents import create_agent

# Inicializar o modelo Databricks
llm = ChatDatabricks(
    endpoint="databricks-gpt-oss-120b",  # ou seu endpoint de serving
    temperature=0.1,
)

# Criar agente com o objeto do modelo
agent = create_agent(
    model=llm,
    tools=[execute_sql],
    system_prompt=SYSTEM_PROMPT,
)

## 📊 Visualizando o Grafo do Agente

O LangChain usa o **LangGraph** por baixo dos panos. Podemos visualizar o grafo do agente para entender como ele funciona.

Este diagrama mostra o ciclo ReAct em ação: o modelo raciocina, chama ferramentas, e decide se pode responder ou precisa continuar.

In [0]:
from IPython.display import Image, display

display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

## 🚀 Executando Queries - Veja o Agente em Ação!

Agora vamos testar o agente. Observe:

1. **O agente NÃO conhece o schema** - ele precisa descobrir sozinho via SQL
2. **O agente pode errar** - mas se autocorrige quando recebe mensagens de erro
3. **Usamos `agent.stream`** - para ver as mensagens em tempo real
4. **O agente NÃO lembra entre invocações** - cada pergunta começa do zero (veremos como resolver isso no Lab 6 - Memory)

### O que observar na saída:
- `HumanMessage`: Sua pergunta
- `AIMessage` com `tool_calls`: O agente decidindo chamar uma ferramenta
- `ToolMessage`: Resultado da ferramenta (dados ou erro)
- `AIMessage` final: Resposta formatada para o usuário

In [0]:
question = "Qual tabela tem maior numero de registros no catalogo nasa_gcn e schema silver?"

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

In [0]:
question = "Liste todas as tabelas disponíveis no catálogo nasa_gcn e schema silver."

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 🎮 Sua Vez - Experimente!

Agora é sua vez de testar o agente. Tente perguntas como:
- "Quantos registros existem na tabela X?"
- "Quais colunas tem a tabela Y?"
- "Liste os valores únicos de uma coluna"
- "Faça um resumo estatístico"

**Dica**: Observe como o agente descobre o schema, trata erros, e formata respostas!

In [0]:
question = "Quais são os tipos de eventos mais frequentes no catálogo nasa_gcn e schema silver?"

for step in agent.stream(
    {"messages": question},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()

## 📚 Resumo e Próximos Passos

### O que aprendemos:
- **LangChain** simplifica a criação de aplicações com LLMs
- **Agentes ReAct** operam em ciclo de raciocínio → ação → observação
- **Tools** permitem ao agente interagir com sistemas externos
- **System Prompts** definem o comportamento do agente
- **MLflow Autolog** registra automaticamente as execuções

### Limitações deste agente básico:
1. **Sem memória** - esquece tudo entre invocações
2. **Sem streaming token-a-token** - só vemos passos completos
3. **Sem controle de acesso** - qualquer query é permitida
4. **Sem aprovação humana** - executa automaticamente

### Nos próximos labs, você aprenderá:
- **L2 - Messages**: Entender os tipos de mensagens (Human, AI, Tool, System)
- **L3 - Streaming**: Receber respostas token por token
- **L4 - Tools**: Criar ferramentas mais sofisticadas
- **L5 - MCP**: Conectar a servidores de ferramentas externos
- **L6 - Memory**: Adicionar memória de curto prazo
- **L7 - Structured Output**: Gerar respostas em formatos específicos
- **L8 - Dynamic Prompts**: Adaptar o prompt dinamicamente
- **L9 - Human in the Loop**: Adicionar aprovação humana